In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_a = pd.read_csv("LVB-75DPI-metadata-feature-table.csv")

data_restore = pd.read_csv("LVB-mass-restored-15.csv")
data_restore_list = data_restore.filename.to_list()
data_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 1:8]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api

df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list)), columns=['col_name','corr', 'corr_pval'])

df_final['corr'].replace('', float("NaN"), inplace=True)
df_final.dropna(subset=['corr'], inplace=True)

dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(list(df_final["corr_pval"]), method='indep')
df_final['FDR_rejected'] = dr_fdr_pval[0]
df_final['FDR_pval'] = dr_fdr_pval[1]

df_final.to_csv('LVB-75DPI-group15-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.weight..Body.weight.ratio, Liver.weight..Body.weight.ratio, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X119.049_2.765, X251.037_0.323, X291.146_2.39, X386.717_2.856, X494.325_2.865, X620.356_2.913]
Index: []
['Heart.weight..Body.weight.ratio-X119.049_2.765', 'Heart.weight..Body.weight.ratio-X251.037_0.323', 'Heart.weight..Body.weight.ratio-X291.146_2.39', 'Heart.weight..Body.weight.ratio-X386.717_2.856', 'Heart.weight..Body.weight.ratio-X494.325_2.865', 'Heart.weight..Body.weight.ratio-X620.356_2.913', 'Liver.weight..Body.weight.ratio-X119.049_2.765', 'Liver.weight..Body.weight.ratio-X251.037_0.323', 'Liver.weight..Body.weight.ratio-X291.146_2.39', 'Liver.weight..Body.weight.ratio-X386.717_2.856', 'Liver.weight..Body.weight.ratio-X494.325_2.865', 'Liver.weight..Body.weight.ratio-X620.356_2.913', 'Ejection.Fraction-X119.049_2.765', 'Ejection.Fraction-X251.037_0.323', 'Ejection.Fraction-